# SETUP ENVIRONMENT

In [23]:
# mnt drive 
from google.colab import drive
drive.mount('/content/drive')


# Access token from drive without exposing in text (accsessible in logfiles lol. Better than nothing)
import os
import subprocess

token = open("/content/drive/MyDrive/token/colab.txt", "r").read()
tkn = token.split("=")

repo_name = "callebalik/clinical_NLP_SE.git"
cmd_string = 'git clone https://{0}@github.com/{1}'.format(tkn[1], repo_name)

!{cmd_string}
cmd_string = "" # removing the variable
print(cmd_string)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'clinical_NLP_SE' already exists and is not an empty directory.



In [25]:
# Set paths for the environment 
import os 
from pathlib import Path

drive_path = Path('/content/drive/MyDrive/bioNLP/')
code_path = Path('/content/clinical_NLP_SE/')
# sys.path.append(str(code_path)) # Let pyhton find scripts from the repo
models_path = Path('/content/drive/My Drive/bioNLP/models/')
data_path = code_path / 'data/'
corpus_path = data_path / 'corpus'

In [ ]:
#drive.flush_and_unmount()
#print('All changes made in this colab session should now be visible in Drive.')

In [ ]:
%load_ext autoreload
%autoreload 2

# GET DATA AND MODELS

## Gettings the Swedish models 

https://data.kb.se/datasets/2020/10/swedish_nlp/spacy/

https://github.com/Kungbib/swedish-spacy

The models can be downloaded with the commands below. For colab this is slow and they are instead mounted from a google drive where they have been downloaded



In [ ]:
# Get models from Kungbib https://github.com/Kungbib/swedish-spacy
#!wget https://data.kb.se/datasets/2020/10/swedish_nlp/spacy/sv_tagger-0.0.0.tar.gz

# Older models
#!wget https://data.kb.se/datasets/2020/10/swedish_nlp/spacy/sv_model_upos.zip
#!wget https://data.kb.se/datasets/2020/10/swedish_nlp/spacy/sv_model_xpos.zip

In [ ]:
# N.B. Make sure to install the spacy-transformers extension, or the models won't work.
!pip install spacy-transformers
!pip install "/content/drive/My Drive/bioNLP/models/sv_pipeline-0.0.0.tar.gz"
import spacy


### Note from Kungbib

** UPDATE February 2021 **: We are adding two transformer-based models trained in spaCy 3.0. They are available to download at the same link given below.
One model is a complete pipeline with UPOS tagger, parser, sentencer, ner and lemmatizer (sv_pipeline-0.0.0.tar.gz). Unfortunately the lemmatizer is not yet trainable in spaCy, so the performance is as good as the quality of the rules/lookup tables available for Swedish (i.e. not very good). 

If you need a Swedish lemmatizer we advise you for the moment to have a look at Stanza, efselab or lemmy.ww

---


The other model is a XPOS tagger in case you need language-specific part-of-speech tags (sv_tagger-0.0.0.tar.gz).

# Analyzing

In [42]:
import pandas as pd
from io import StringIO

txt = """HEADING1
value 1

HEADING2
value 2

HEADING1
value 11

HEADING2
value 12"""

cols, vals = zip(*[line.split('\n') for line in StringIO(txt).read().split('\n\n')])

s = pd.Series(vals, cols)
s.index = [s.groupby(level=0).cumcount(), s.index]

s.unstack()

,HEADING1,HEADING2
0,value 1,value 2
1,value 11,value 12


In [40]:
import pandas as pd 

file = open(data_path / 'raw/negations/negEx.txt')
data = pd.read_csv(file, sep=" ", header=None)
# data.columns = ["a", "b", "c", "etc."]

ParserError: ignored

In [ ]:
import spacy 
nlp = spacy.load("sv_pipeline")

Jag             PRON            nsubj          
gillar          VERB            ROOT           
London          PROPN           obj            
och             CCONJ           cc             
Berlin          PROPN           conj           
.               PUNCT           punct          
London LOC
Berlin LOC


In [90]:
import pandas as pd
import re 

neg_terms = {}
with open(data_path / 'raw/negations/negEx.txt','r') as negations:
    for line in negations:
        x = line.split("\t")
        key = x[0] 
        value = x[1].strip()
        neg_terms[key] = value
    
df = pd.DataFrame.from_dict(neg_terms, orient="index")
df.columns()
df.sort_index()



,0
aldrig,[pren]
avsaknad av,[pren]
ej,[pren]
fri från,[pren]
förnekar,[pren]
förnekas,[post]
har inte,[pren]
icke,[pren]
inga,[pren]
inga tecken,[pren]


In [35]:
# Negations 
neg_terms = []

with open(data_path / 'raw/negations/negEx.txt','r') as negations:
      negs = dict((k.strip(), v.strip()) for k,v in 
              (negations.split(' [') for item in s.split('\t')))
      print(neg_terms)

NameError: ignored

In [27]:
file =  corpus_path / 'mockup-patient-records/by_johanna/originals/chart1.txt'


pre_doc = open(file, 'r').read()

doc = nlp(pre_doc)
width = 20 # table width
for token in doc:
    print(f"{token.text: <{width}} {token.tag_: <{width}} {token.dep_: <{width}}")

Kontaktorsak         NOUN                 ROOT                
:                    PUNCT                punct               
Dyspné               NOUN                 ROOT                

                    PUNCT                ROOT                
Tidigare             ADJ                  amod                
sjukdomar            NOUN                 ROOT                
:                    PUNCT                punct               
Insulin              ADJ                  amod                
och                  CCONJ                cc                  
tablettbehandlad     ADJ                  conj                
DM2                  NUM                  ROOT                
.                    PUNCT                punct               
Tablettbehandlad     ADJ                  amod                
hypertoni            NOUN                 ROOT                
.                    PUNCT                punct               
Höga                 ADJ                  amod         

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Tidigare TME
för tre dagar sedan TME
under gårdagen TME
Idag TME
Idag TME
92% MSR
imorgon TME


In [ ]:
import pandas as pd

cols = ("text", "lemma", "POS", "explain", "stopword")
rows = []

for t in doc:
    row = [t.text, t.lemma_, t.pos_, spacy.explain(t.pos_), t.is_stop]
    rows.append(row)

df = pd.DataFrame(rows, columns=cols)
    
df



,text,lemma,POS,explain,stopword
0,Kontaktorsak,Kontaktorsak,NOUN,noun,False
1,:,:,PUNCT,punctuation,False
2,Dyspné,Dyspné,NOUN,noun,False
3,\n,\n,PUNCT,punctuation,False
4,Tidigare,Tidigare,ADJ,adjective,True
...,...,...,...,...,...
298,:,:,PUNCT,punctuation,False
299,Pneumoni,Pneumoni,NOUN,noun,False
300,",",",",PUNCT,punctuation,False
301,ospecificerad,ospecificerad,ADJ,adjective,False


* raw text
* lemma – a root form of the word
* part of speech
* a flag for whether the word is a stopword – i.e., a common word that may be filtered out

In [ ]:
from spacy import displacy

displacy.render(doc, style="dep", jupyter=True)

In [ ]:
for sent in doc.sents:
    print(">", sent)

In [ ]:
for chunk in doc.noun_chunks:
    print(chunk.text)

In [ ]:
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
displacy.render(doc, style="ent", jupyter=True)

/usr/local/lib/python3.7/dist-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
